# SPDC Source Simulation
## The source node
This example will simulate the laser light source for entangled photons (via SPDC).

The SPDCLightSource component acts as a simple low intensity laser with an SPDC lens. It provides entangled photon clusters at a set frequency.


Attributes:
        name (str): label for the source instance
        timeline (Timeline): timeline for simulation
        frequency (float): frequency (in Hz) of photon creation.
        wavelengths (List[float]): wavelengths (in nm) of emitted entangled photons.
        linewidth (float): st. dev. in photon wavelength (in nm) (currently unused).
        mean_photon_num (float): mean number of photons emitted each period.
        encoding_type (Dict): encoding scheme of emitted photons (as defined in the encoding module).
        phase_error (float): phase error applied to qubits.



In [10]:
from sequence.kernel.timeline import Timeline
from sequence.topology.node import Node
from sequence.components.memory import Memory

class source(Node):
    def __init__(self, name, timeline):
        super().__init__(name, timeline)
        
        memory_name = name + ".memory"
        memory = Memory(memory_name, timeline, fidelity=1, frequency=0,
                        efficiency=1, coherence_time=0, wavelength=500)
        self.add_component(memory)
        memory.add_receiver(self)

    def get(self, photon, **kwargs):
        self.send_qubit(kwargs['dst'], photon)
        
class Counter:
    def __init__(self):
        self.count = 0
        self.time = 0

    def trigger(self, detector, info):
        self.count += 1
        self.time = info['time']
        
class receiver(Node):
    def __init__(self, name, timeline):
        super().__init__(name, timeline)

        detector_name = name + ".detector"
        detector = Detector(detector_name, timeline, efficiency=1)
        self.add_component(detector)
        self.set_first_component(detector_name)
        detector.owner = self

        self.counter = Counter()
        detector.attach(self.counter)

    def receive_qubit(self, src, qubit):
        self.components[self.first_component_name].get()


## Build the Network.
We are now ready to start writing the main function of our script. The first step is to create the simulation timeline. We will use a 10 second run time, but more or less time may be needed depending on hardware parameters. Note that the runtime is given in picoseconds. 
Then we will need to create the source node by specifying a name and the timeline it belongs to.

In [42]:
from sequence.kernel.timeline import Timeline
tl = Timeline(10e12) #10 seconds in picosecond

my_source = source("my_source",tl)
my_detector = receiver("my_detector",tl)
#Note that we also set the random generator seed for our node to ensure reproducability.
my_source.set_seed(0)
my_detector.set_seed(1) 

#create the quantum channel and note that We won’t need a classical channel,
#as we’re not sending any messages between nodes.
from sequence.components.optical_channel import QuantumChannel

qc = QuantumChannel("qc", tl, attenuation=0, distance=1e3)
qc.set_ends(my_source, my_detector.name)

# Measure the Memory Once
With the network built, we are ready to schedule simulation events and run our experiment. The details on scheduling events are covered in Tutorial 1, so we will not focus on them here. Let’s first run one experiment with the memory in the |↑⟩ state and observe the detection time of the single emitted photon. We can obtain the memory object using the Node.get_components_by_type method, which returns a list of matching components on the node. The memory state can then be set with the update_state method.

In [48]:
memories = my_source.get_components_by_type("Memory")
memory = memories[0]
memory.update_state([complex(0), complex(1)])

We must also schedule an excite event for the memory, which will send a photon to a connected node supplied as an argument (in this case, we’ll use "node2"). Let’s put it at time 0:

In [44]:
from sequence.kernel.process import Process
from sequence.kernel.event import Event

process = Process(memory, "excite", ["my_detector"])
event = Event(0, process)
tl.schedule(event)

In [47]:
tl.init()
tl.run()
print("detection count: {}".format(my_detector.counter.count))
print("detection time: {} ps".format(my_detector.counter.time))
print("delay = L / c = {} ps".format(1e3*1e12/(299792458)))

detection count: 1
detection time: 4999950 ps
delay = L / c = 3335640.9519815203 ps
